In [1]:
import csv
from collections import deque
import pandas as pd

In [2]:
def calculate_capital_gains(input_csv_path, output_csv_path, crypto_symbols, quote_currency=None):
    # LIFO stack for keeping track of crypto purchases
    stack = deque()
    total_gains_crypto = 0

    with open(input_csv_path, mode='r') as file:
        reader = csv.DictReader(file)
        rows = list(reader)
        fieldnames = reader.fieldnames + ["Plusvalenze"]

    # Filtra per crypto equivalenti e valuta di scambio
    rows = [
        row for row in rows
        if row["base"] in crypto_symbols and (quote_currency is None or row["quote"] == quote_currency)
    ]

    # Se non ci sono righe per questa combinazione, termina senza fare nulla
    if not rows:
        # print(f"Nessuna transazione trovata per {'/'.join(crypto_symbols)}-{quote_currency}.")
        return 0

    for row in rows:
        row["Plusvalenze"] = 0  # Default plusvalenza to 0

        volume = float(row["volume"])  # Crypto amount
        amount = float(row["amount"])  # Currency amount (EUR, USDC, etc.)
        rate = float(row["rate"])  # Price per crypto unit

        if row["side"] == "BUY":
            # Add the purchase to the LIFO stack
            stack.append({"volume": volume, "rate": rate})

        elif row["side"] == "SELL":
            sell_volume = volume
            gain = 0

            # Process LIFO stack for matching volumes
            while sell_volume > 0 and stack:
                last_purchase = stack.pop()
                available_volume = last_purchase["volume"]
                purchase_rate = last_purchase["rate"]

                if available_volume <= sell_volume:
                    # Consume entire purchase
                    gain += available_volume * (rate - purchase_rate)
                    sell_volume -= available_volume
                else:
                    # Consume part of the purchase
                    gain += sell_volume * (rate - purchase_rate)
                    last_purchase["volume"] -= sell_volume
                    stack.append(last_purchase)  # Push back remaining tokens
                    sell_volume = 0

            row["Plusvalenze"] = round(gain, 2)
            total_gains_crypto += gain

    # Write the updated rows with the new column to the output CSV
    with open(output_csv_path, mode='w', newline='') as file:
        writer = csv.DictWriter(file, fieldnames=fieldnames)
        writer.writeheader()
        writer.writerows(rows)

    print(f"Totale plusvalenze generate per {'/'.join(crypto_symbols)}-{quote_currency}: {round(total_gains_crypto, 2)}")
    return total_gains_crypto

In [3]:
# Esempio di utilizzo
input_file_path = r"C:\Users\Alessandro\Downloads\Report ordini YP 20 11 24.csv"  # Nome del file CSV di input

file = pd.read_csv(input_file_path)

# Configurazione delle alternative names
alternative_names = [
    ['RNDR', 'RENDER']
]

total_gains = 0
gains_by_quote = {}  # Dizionario per tracciare i guadagni totali per ogni quote

# Trova tutte le valute base e di scambio
unique_cryptos = file["base"].unique()
unique_quotes = file["quote"].unique()

for quote_currency in unique_quotes:
    processed_groups = set()  # Per tracciare i gruppi già calcolati
    total_gains_for_quote = 0  # Per tracciare i guadagni totali di questa quote

    for crypto in unique_cryptos:
        # Trova il gruppo equivalente per la crypto
        relevant_group = None
        for group in alternative_names:
            if crypto in group:
                relevant_group = group
                break

        # Determina il gruppo da utilizzare
        if relevant_group:
            group_key = tuple(sorted(relevant_group))
            if group_key in processed_groups:
                continue  # Evita di calcolare di nuovo lo stesso gruppo
            processed_groups.add(group_key)
            crypto_symbols = relevant_group
        else:
            crypto_symbols = [crypto]  # Se non è in un gruppo, usa il nome unico

        # Calcola le plusvalenze
        output_file_path = rf"C:\Users\Alessandro\Downloads\{'_'.join(crypto_symbols)}_{quote_currency}_plusvalenze.csv"
        total_gains_crypto = calculate_capital_gains(
            input_file_path, output_file_path, crypto_symbols=crypto_symbols, quote_currency=quote_currency
        )

        # Aggiungi al totale delle plusvalenze solo se ci sono transazioni
        if total_gains_crypto > 0:
            total_gains_for_quote += total_gains_crypto
            total_gains += total_gains_crypto

    # Salva i guadagni totali per questa quote
    gains_by_quote[quote_currency] = round(total_gains_for_quote, 2)

print('')
# Stampa i totali per ogni valuta di scambio
for quote_currency, gains in gains_by_quote.items():
    print(f"Totale plusvalenze generate in {quote_currency}: {gains}")

Totale plusvalenze generate per BTC-EUR: 249.36
Totale plusvalenze generate per RNDR/RENDER-EUR: 19.77
Totale plusvalenze generate per LINK-EUR: 0.18
Totale plusvalenze generate per SEI-EUR: 0.55
Totale plusvalenze generate per TIA-EUR: 1.22
Totale plusvalenze generate per IMX-EUR: 1.21
Totale plusvalenze generate per FET-EUR: 8.08
Totale plusvalenze generate per ETH-EUR: 0
Totale plusvalenze generate per USDC-EUR: 0
Totale plusvalenze generate per FET-USDC: 0

Totale plusvalenze generate in EUR: 280.36
Totale plusvalenze generate in USDC: 0


In [4]:
file_btc = pd.read_csv(r'C:\Users\Alessandro\Downloads\BTC_EUR_plusvalenze.csv')
file_btc_sell = file_btc[file_btc["side"] == "SELL"]
file_btc_sell

,id,base,quote,amount,volume,rate,brokerage,brokerage_currency,side,date,Plusvalenze
54,61120546,BTC,EUR,9.831018,0.00016,61443.86,0.029493,EUR,SELL,2024-05-19T13:03:37.617Z,0.66
55,61130369,BTC,EUR,30.586463,0.00047,65077.58,0.091759,EUR,SELL,2024-05-21T13:30:08.083Z,3.81
56,61130370,BTC,EUR,16.894688,0.00026,64979.57,0.050684,EUR,SELL,2024-05-21T13:30:08.083Z,2.40
62,61211502,BTC,EUR,17.130201,0.00028,61179.29,0.051391,EUR,SELL,2024-06-20T10:36:22.507Z,0.24
66,61240323,BTC,EUR,21.721683,0.00037,58707.25,0.065165,EUR,SELL,2024-07-01T06:58:44.15Z,0.42
77,61884500,BTC,EUR,11.996054,0.00016,74975.34,0.035988,EUR,SELL,2024-11-10T18:08:27.963Z,3.02
78,61884501,BTC,EUR,11.983939,0.00016,74899.62,0.035952,EUR,SELL,2024-11-10T18:08:27.963Z,3.01
79,61884502,BTC,EUR,208.218720,0.00278,74898.82,0.624656,EUR,SELL,2024-11-10T18:08:27.963Z,54.53
80,61947845,BTC,EUR,49.951485,0.00058,86123.25,0.149854,EUR,SELL,2024-11-17T10:01:08.357Z,20.49
81,61947846,BTC,EUR,418.054041,0.00486,86019.35,1.254162,EUR,SELL,2024-11-17T10:01:08.357Z,160.79
